In [3]:
from gurobipy import *
import numpy as np
import math as mt

In [4]:
# Données
service_count=2
project_count=4
n = 2
p=4
budget_limit = 100
constraint_count=n*n +1
variable_count= n*n + n  + p
couts_projet = np.array([40, 50, 60, 50])

In [5]:
# Matrice d'utilte
matrice_utilite = np.array([[19, 6, 17, 2], [2, 11, 4, 18]])
print(matrice_utilite)

[[19  6 17  2]
 [ 2 11  4 18]]


In [6]:
# vecteur w
w = [10, 1]

# creer le vecteur w'
w_prime = [(w[k] - w[k+1]) for k in range(len(w)-1)]
w_prime.append(w[-1])
print(w_prime)

[9, 1]


In [7]:
# matrice utilite
u = matrice_utilite

In [8]:
# Generation de la matrice des contraintes 

In [9]:
## les variables rk
coef_rk = []
for k in range(n):
        for i in range(n):
                coef_rk.append([])
                for j in range(n):
                        coef_rk[-1].append(0)
                coef_rk[-1][k] = 1
coef_rk = np.array(coef_rk)
print(coef_rk)


[[1 0]
 [1 0]
 [0 1]
 [0 1]]


In [10]:
## les variables bik
coef_bik = -1 * np.identity(n**2)
print(coef_bik)

[[-1. -0. -0. -0.]
 [-0. -1. -0. -0.]
 [-0. -0. -1. -0.]
 [-0. -0. -0. -1.]]


In [11]:
## les variables x
coef_xi = []
for k in range(n**2):
        coef_xi.append([])
        for i in range(p):
                coef_xi[-1].append(-u[k%n][i])
print(coef_xi)
coef_xi = np.array(coef_xi)

[[-19, -6, -17, -2], [-2, -11, -4, -18], [-19, -6, -17, -2], [-2, -11, -4, -18]]


In [12]:
# contrainte sur le budget
budget = [0 for i in range(n*(n+1))]
budget += [couts_projet[i] for i in range(p)]
budget = np.array(budget)
print(budget)

[ 0  0  0  0  0  0 40 50 60 50]


In [13]:
# concatenation des matrices rk, bik, xk 
result_concat = np.concatenate(
        (coef_rk, np.concatenate((coef_bik, coef_xi),axis=1)),
        axis=1)

# ajout de la contrainte sur les budgets
constrainte = np.vstack([result_concat, budget])
print(constrainte)


[[  1.   0.  -1.  -0.  -0.  -0. -19.  -6. -17.  -2.]
 [  1.   0.  -0.  -1.  -0.  -0.  -2. -11.  -4. -18.]
 [  0.   1.  -0.  -0.  -1.  -0. -19.  -6. -17.  -2.]
 [  0.   1.  -0.  -0.  -0.  -1.  -2. -11.  -4. -18.]
 [  0.   0.   0.   0.   0.   0.  40.  50.  60.  50.]]


In [14]:
## second memebre
second_membre = [0 for _ in range(n**2)]
second_membre.append(budget_limit)
second_membre = np.array(second_membre)
print (second_membre)

[  0   0   0   0 100]


In [15]:
# Coefficient de la fonction objectif
## vecteur w'
w_prime = [(w[k] - w[k+1]) for k in range(len(w)-1)]
w_prime.append(w[-1])

## coefficient fonction objectif
c = [(i+1) * w_prime[i] for i in range(n)] # coef des r_k
c += [- w_prime[i%n]  for i in range(n*n)] # coef des b_ik
c += [0 for i in range(p)] # ajout des coef de x
c = np.array(c)
print (c)

[ 9  2 -9 -1 -9 -1  0  0  0  0]


In [16]:
# Creer les models de la fonction objectif
m = Model("exercice1.3")

Restricted license - for non-production use only - expires 2024-10-28


In [17]:
# declaration des variables de decisions

x = []
for i in range(n):
    x.append(m.addVar(vtype=GRB.CONTINUOUS, lb=0, name="r%d" % (i+1)))
for i in range (n):
    for j in range(n):
        x.append(m.addVar(vtype=GRB.CONTINUOUS, lb=0, name="b%d%d" % ((j+1,(i+1)))))
for j in range(p):
    x.append(m.addVar(vtype=GRB.BINARY, lb=0, name="x%d" % (j+1) ))

In [18]:
# maj du modele pour integrer les nouvelles variables
m.update()
print(x)

[<gurobi.Var r1>, <gurobi.Var r2>, <gurobi.Var b11>, <gurobi.Var b21>, <gurobi.Var b12>, <gurobi.Var b22>, <gurobi.Var x1>, <gurobi.Var x2>, <gurobi.Var x3>, <gurobi.Var x4>]


In [19]:
# definition de l'objectif
obj = LinExpr();
obj =0
for j in range(variable_count):
    obj += c[j] * x[j]

m.setObjective(obj,GRB.MAXIMIZE)

In [20]:
# Definition des contraintes
# Definition des contraintes
lignes = range(constraint_count)
colonnes = range(variable_count)
print (lignes, colonnes)
for i in lignes:
    m.addConstr(quicksum(constrainte[i][j]*x[j] for j in colonnes) <= second_membre[i], "Contrainte%d" % i)

range(0, 5) range(0, 10)


In [21]:
# Resolution
m.optimize()


print("")                
print('Solution optimale:')
mvars = m.getVars()
names = m.getAttr('VarName', mvars)
values = m.getAttr('X', mvars)
result = dict(zip(names, values))
for key in result:
    print(key,'=',result[key])
print("")
print('Valeur de la fonction objectif :', m.objVal)


Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (linux64)

CPU model: Intel(R) Core(TM) i7-10510U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 5 rows, 10 columns and 28 nonzeros
Model fingerprint: 0xab4ab079
Variable types: 6 continuous, 4 integer (4 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+01]
  Objective range  [1e+00, 9e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+02, 1e+02]
Found heuristic solution: objective -2.00000e+31
Presolve removed 0 rows and 2 columns
Presolve time: 0.00s

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: -2e+31 
No other solutions better than 0

Model is unbounded
Best objective -2.000000000000e+31, best bound -, gap -

Solution optimale:
r1 = 0.0
r2 = 0.0
b11 = 1e+30
b21 = 1e+30
b12 = 1e+30
b22 = 1e+30
x1 = -0.0
x2 = -0.0
x3 = -0